In [2]:
import sys
sys.path.insert(0,'/Ai/khaled/Ciena-Gen-AI')

import gradio as gr
import os
from backend.llm.baseLLM import Remote_LLM 
# Chatbot demo with multimodal input (text, markdown, LaTeX, code blocks, image, audio, & video). Plus shows support for streaming text.
from backend.retrieval.ciena_retreival import CienaRetrieval
from backend.embedder.baseEmbedder import baseEmbedder
from backend.retrieval.utils import *
from backend.retrieval.rereanker import Reranker
from langchain.document_loaders import JSONLoader

import asyncio
import time
from concurrent.futures import ThreadPoolExecutor


/opt/miniconda3/envs/ciena/lib/python3.11/site-packages/transformers/utils/hub.py:123: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [3]:
embedding_function = baseEmbedder().embedding_function
retriael_kwargs = {
    "threshold": "0.8",
    "k": 20,
    "embedder": embedding_function,
    "hybrid": True,
}
ciena_retreival = CienaRetrieval(**retriael_kwargs)
reranker = Reranker()

.gitattributes: 100%|██████████| 1.18k/1.18k [00:00<00:00, 11.9MB/s]
1_Pooling/config.json: 100%|██████████| 190/190 [00:00<00:00, 1.56MB/s]
README.md: 100%|██████████| 10.6k/10.6k [00:00<00:00, 48.1MB/s]
config_sentence_transformers.json: 100%|██████████| 116/116 [00:00<00:00, 1.06MB/s]
data_config.json: 100%|██████████| 39.3k/39.3k [00:00<00:00, 10.4MB/s]
pytorch_model.bin: 100%|██████████| 90.9M/90.9M [00:01<00:00, 58.1MB/s]
sentence_bert_config.json: 100%|██████████| 53.0/53.0 [00:00<00:00, 601kB/s]
special_tokens_map.json: 100%|██████████| 112/112 [00:00<00:00, 1.20MB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 10.3MB/s]
tokenizer_config.json: 100%|██████████| 350/350 [00:00<00:00, 883kB/s]
train_script.py: 100%|██████████| 13.2k/13.2k [00:00<00:00, 11.9MB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 6.67MB/s]
modules.json: 100%|██████████| 349/349 [00:00<00:00, 899kB/s]


In [7]:
def load_db():
    """Load Ciena database."""
    
    dir = '/Ai/khaled/Ciena-Gen-AI/output/'
    loaded_data = []
    for r, d, f in os.walk(dir):
        
        for file in f:
            # if "10Aug-BP_Engineering_guide" in file or "Blue_Planet_MLA_Cloud_Deployment_Guide" in file:
            if '.json' in file and file != 'structuredData.json':
                dir_ = file.split('.')[0] + '.pdf'
                file_name = os.path.join(dir, dir_, file)
                print(file_name)
                try:
                    loader = JSONLoader(
                        file_path=file_name,
                        jq_schema='.[].content[]',
                        content_key="text", 
                        metadata_func=metadata_func)

                    loaded_data.extend(loader.load())
                    print(f"Successfully loaded file {file_name}")
                except Exception as e:
                    # import pdb; pdb.set_trace()
                    print(f"error in loading  file {file_name}")
                    print(e)

    return loaded_data


In [8]:
db = load_db()

/Ai/khaled/Ciena-Gen-AI/output/17-Dec-UAA_21.pdf/17-Dec-UAA_21.json
error in loading  file /Ai/khaled/Ciena-Gen-AI/output/17-Dec-UAA_21.pdf/17-Dec-UAA_21.json
[Errno 2] No such file or directory: '/Ai/khaled/Ciena-Gen-AI/output/17-Dec-UAA_21.pdf/17-Dec-UAA_21.json'
/Ai/khaled/Ciena-Gen-AI/output/10Aug-BP_Engineering_guide.pdf/10Aug-BP_Engineering_guide.json
Successfully loaded file /Ai/khaled/Ciena-Gen-AI/output/10Aug-BP_Engineering_guide.pdf/10Aug-BP_Engineering_guide.json
/Ai/khaled/Ciena-Gen-AI/output/uaa-admin-guide.pdf/uaa-admin-guide.json
error in loading  file /Ai/khaled/Ciena-Gen-AI/output/uaa-admin-guide.pdf/uaa-admin-guide.json
Expected page_content is string, got <class 'dict'> instead.                     Set `text_content=False` if the desired input for                     `page_content` is not a string
/Ai/khaled/Ciena-Gen-AI/output/5G_Netwrok_Slicing_Installation_Guide_21.pdf/5G_Netwrok_Slicing_Installation_Guide_21.json
error in loading  file /Ai/khaled/Ciena-Gen-AI/out

In [9]:
db

[Document(page_content='Blue Planet Engineering Guide ', metadata={'source': '/Ai/khaled/Ciena-Gen-AI/output/10Aug-BP_Engineering_guide.pdf/10Aug-BP_Engineering_guide.json', 'seq_num': 1, 'header': '', 'table_path': '', 'page_number': 0, 'type': 'P'}),
 Document(page_content='Blue Planet Engineering 22.08 ', metadata={'source': '/Ai/khaled/Ciena-Gen-AI/output/10Aug-BP_Engineering_guide.pdf/10Aug-BP_Engineering_guide.json', 'seq_num': 2, 'header': '', 'table_path': '', 'page_number': 0, 'type': 'P[2]'}),
 Document(page_content='2022: Draft ', metadata={'source': '/Ai/khaled/Ciena-Gen-AI/output/10Aug-BP_Engineering_guide.pdf/10Aug-BP_Engineering_guide.json', 'seq_num': 3, 'header': '', 'table_path': '', 'page_number': 0, 'type': 'P[3]'}),
 Document(page_content='Publication history ', metadata={'source': '/Ai/khaled/Ciena-Gen-AI/output/10Aug-BP_Engineering_guide.pdf/10Aug-BP_Engineering_guide.json', 'seq_num': 4, 'header': 'Table of Contents ', 'table_path': '', 'page_number': 1, 'type':

In [10]:
def clean(docs):
    loaded_data = filter_empty(docs)
    loaded_data = filter_redundant(loaded_data)
    loaded_data = exclude_toc(loaded_data)
    return loaded_data


In [11]:
def get_relevant_docs(query, docs):
    """Get relevant documents from Ciena database."""
    if len(query) == 0 or len(docs) == 0:
        return []
    ciena_retrieval = CienaRetrieval(**retriael_kwargs)
    relevant_docs = ciena_retrieval.get_res(query, docs)
    reranked_res = reranker.rerank(query, relevant_docs)
    return reranked_res

In [12]:
def get_context(docs, headers):
    if len(headers) == 0:
        return [], []
    context, sources = ciena_retreival.get_context(docs, headers)
    return context, sources


In [13]:
def add_text(history, text):
    history = history + [(text, None)]
    return history, gr.Textbox(value="", interactive=False)

def bot(history):
    response = "**That's cool!**"
    history[-1][1] = ""
    for character in response:
        history[-1][1] += character
        time.sleep(0.05)
        yield history

In [14]:
def main_get_src_ctx(message, seconds):
    query = message
    relevant_docs = get_relevant_docs(query, cleaned_db)
    rel_headers = relevant_headers(relevant_docs)
    rel_headers = [x for x in rel_headers if x != 'Table of Contents ']
    context, sources = get_context(loaded_db, rel_headers)
    return context, sources


In [15]:
def gt_llm_answer(question, ctx, src):
    endpoint = " http://0.0.0.0:8000/answer"
    LLM_kwargs={'max_new_tokens': 500, 'temperature': 0.5}

    llm = Remote_LLM(
        endpoint="http://0.0.0.0:8000/answer",
        generation_config=LLM_kwargs
    )
    ctx = ctx[len(ctx) // 2:]
    if len(ctx) > 2000: 
        ctx = ctx[:2000]
    prompt = f"""
    You are a powerful AI asistant that answers only based on the given contex. If the context is not enough, you can ask for more information.
    Given the following context {ctx}, answer the following question: {question}
    """

    answer = llm(prompt)
    return answer, src



In [16]:
def slow_echo(message, history):
    ctx, src = main_get_src_ctx(message, 3)
    # convert list ctx to string
    ctx =' '.join(ctx)
    answer, src = gt_llm_answer(message, ctx, src)
    return answer #

In [17]:
loaded_db = load_db()
cleaned_db = clean(loaded_db)

/Ai/khaled/Ciena-Gen-AI/output/17-Dec-UAA_21.pdf/17-Dec-UAA_21.json
error in loading  file /Ai/khaled/Ciena-Gen-AI/output/17-Dec-UAA_21.pdf/17-Dec-UAA_21.json
[Errno 2] No such file or directory: '/Ai/khaled/Ciena-Gen-AI/output/17-Dec-UAA_21.pdf/17-Dec-UAA_21.json'
/Ai/khaled/Ciena-Gen-AI/output/10Aug-BP_Engineering_guide.pdf/10Aug-BP_Engineering_guide.json
Successfully loaded file /Ai/khaled/Ciena-Gen-AI/output/10Aug-BP_Engineering_guide.pdf/10Aug-BP_Engineering_guide.json
/Ai/khaled/Ciena-Gen-AI/output/uaa-admin-guide.pdf/uaa-admin-guide.json
error in loading  file /Ai/khaled/Ciena-Gen-AI/output/uaa-admin-guide.pdf/uaa-admin-guide.json
Expected page_content is string, got <class 'dict'> instead.                     Set `text_content=False` if the desired input for                     `page_content` is not a string
/Ai/khaled/Ciena-Gen-AI/output/5G_Netwrok_Slicing_Installation_Guide_21.pdf/5G_Netwrok_Slicing_Installation_Guide_21.json
error in loading  file /Ai/khaled/Ciena-Gen-AI/out

In [18]:
len(cleaned_db)

2269

In [118]:
query = "How to cook an egg?"#"give me a table for ciena's BPO Runtime License"


In [119]:
relevant_docs = get_relevant_docs(query, cleaned_db)
rel_headers = relevant_headers(relevant_docs)
rel_headers = [x for x in rel_headers if x != 'Table of Contents ']
context, sources = get_context(loaded_db, rel_headers)

In [121]:
context

[]

In [122]:
def remove_duplicates_preserve_order(seq):
    seen = set()
    return [x for x in seq if not (x in seen or seen.add(x))]

ctx = remove_duplicates_preserve_order(context)
ctx = '\n'.join(ctx)
print(ctx)

In [25]:
# DRAGON
# full_prompt = f"""<human>: You are a helpful assistant who speaks with facts and no wasted words.
# Please read the following text:

# Context-1: {ctx}

# Please answer the question: 
# {query}

# Please only use facts in the text. If the text does not provide the answer, then please respond with: {{not_found_response}}

# <bot>: """


In [123]:
print(ctx)

In [114]:
query = 'Who is Microsoft CEO?'

In [140]:
ctx = ""

In [141]:
full_prompt = f"""\
<|system|> Given a part of a lengthy markdown document, answer the following question: `{query}`. Please, follow the same format as the source document given. </s>
<|user|>
please ONLY respond with: {{not_found_response}}, if the context does not provide the answer </s>
CONTEXT: {ctx} 

<|assistant|> """

In [142]:
endpoint = " http://0.0.0.0:8000/answer"
LLM_kwargs={'max_new_tokens': 1500, 'temperature': 0.4}

llm = Remote_LLM(
        endpoint="http://0.0.0.0:8000/answer",
        generation_config=LLM_kwargs
    )

In [143]:
answer = llm(full_prompt)
bot_response = answer.split('<|assistant|>')[1].split('</s>')[0]
print(bot_response)

with open('output.txt', 'w') as f:
    f.write(bot_response)

 
<bot>:Not Found.


In [63]:
from transformers import AutoTokenizer, AutoModelForCausalLM  
tokenizer = AutoTokenizer.from_pretrained("llmware/dragon-mistral-7b-v0")  

In [105]:

from transformers import AutoTokenizer, T5ForConditionalGeneration

tokenizer = AutoTokenizer.from_pretrained("llmware/dragon-mistral-7b-v0")
tokenized = tokenizer(bot_response, return_tensors="pt")
print(len(tokenized["input_ids"][0]))

678


In [58]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt2")
num_tokens = len(encoding.encode(full_prompt))
num_tokens


841